In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# read in the csv result from xgboost
#xgb_sub = pd.read_csv('../input/xgboost/submission (2).csv')
xgb_sub = pd.read_csv('submission (2).csv')

In [5]:
# add log score for the winPlacePerc
xgb_sub['log1']=xgb_sub['winPlacePerc'].apply(lambda x: np.log(x))

In [6]:
# read in the csv result from lightgbm
#light_sub = pd.read_csv('../input/lightgbm/submission (1).csv')
light_sub = pd.read_csv('submission (1).csv')

In [7]:
# add log score for the winPlacePerc
light_sub['log2']=light_sub['winPlacePerc'].apply(lambda x: np.log(x))

In [10]:
# read in the csv result from catboost
#cat_sub = pd.read_csv('../input/catboost/submission_pubg.csv')
cat_sub = pd.read_csv('cat_sub.csv')

In [11]:
# add log score for the winPlacePerc
cat_sub['log3']=cat_sub['winPlacePerc'].apply(lambda x: np.log(x))

In [12]:
#take a lot at the data
xgb_sub.head()

,Id,winPlacePerc,log1
0,9329eb41e215eb,0.238785,-1.432191
1,639bd0dcd7bda8,0.918271,-0.085262
2,63d5c8ef8dfe91,0.753214,-0.283406
3,cf5b81422591d1,0.538126,-0.619663
4,ee6a295187ba21,0.924251,-0.078772


In [13]:
#take a lot at the data
light_sub.head()

,Id,winPlacePerc,log2
0,9329eb41e215eb,0.259259,-1.349927
1,639bd0dcd7bda8,0.936170,-0.065958
2,63d5c8ef8dfe91,0.666667,-0.405465
3,cf5b81422591d1,0.545455,-0.606136
4,ee6a295187ba21,0.925926,-0.076961


In [14]:
#take a lot at the data
cat_sub.head()

,Id,winPlacePerc,log3
0,9329eb41e215eb,0.249395,-1.388715
1,639bd0dcd7bda8,0.955871,-0.045133
2,63d5c8ef8dfe91,0.660827,-0.414263
3,cf5b81422591d1,0.533029,-0.629179
4,ee6a295187ba21,0.920996,-0.082299


In [15]:
#merge them into one dataset
dfs = [xgb_sub, light_sub, cat_sub]

In [16]:
#import function tools for reduce
import functools

In [17]:
# reduce lambda
df_final = functools.reduce(lambda left,right: pd.merge(left,right,on='Id'), dfs)

In [18]:
# take a look at the final dataset
df_final.head()

,Id,winPlacePerc_x,log1,winPlacePerc_y,log2,winPlacePerc,log3
0,9329eb41e215eb,0.238785,-1.432191,0.259259,-1.349927,0.249395,-1.388715
1,639bd0dcd7bda8,0.918271,-0.085262,0.936170,-0.065958,0.955871,-0.045133
2,63d5c8ef8dfe91,0.753214,-0.283406,0.666667,-0.405465,0.660827,-0.414263
3,cf5b81422591d1,0.538126,-0.619663,0.545455,-0.606136,0.533029,-0.629179
4,ee6a295187ba21,0.924251,-0.078772,0.925926,-0.076961,0.920996,-0.082299


In [19]:
#create a empty data frame for the final submission
en_sub = pd.DataFrame()

In [20]:
# get id from one of the dataset
en_sub['Id'] = cat_sub['Id']